Creating the dataset

In [ ]:
import cv2
import numpy as np
import os
import random

# load the apple image
img = cv2.imread('/content/apple.jpg')

# get the size of the apple image
apple_size = img.shape[0]

# create directories for each quadrant
quadrant_names = ['quadrant_0', 'quadrant_1', 'quadrant_2', 'quadrant_3']
for quadrant_name in quadrant_names:
    os.makedirs(quadrant_name, exist_ok=True)

# create 100 images for each quadrant
quadrants = [(0, 0), (200, 0), (0, 200), (200, 200)]
for i, q in enumerate(quadrants):
    x, y = q
    for j in range(100):
        quadrant_img = np.zeros((400, 400, 3), np.uint8)
        
        # randomize the position of the apple image in the quadrant
        dx = random.randint(0, 200 - apple_size)
        dy = random.randint(0, 200 - apple_size)
        quadrant_img[y+dy:y+dy+apple_size, x+dx:x+dx+apple_size] = img
        
        filename = f"apple_{i}_{j}.jpg"
        filepath = os.path.join(quadrant_names[i], filename)
        cv2.imwrite(filepath, quadrant_img)


CNN Model

In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models

# set the directory containing the apple images
directory = 'apple_images'

# load the apple images and their labels
images = []
labels = []
for i in range(4):
    for j in range(100):
        filename = f"apple_{i}_{j}.jpg"
        filepath = os.path.join(directory, f"quadrant_{i}", filename)
        img = cv2.imread(filepath)
        images.append(img)
        labels.append(i)

# convert the images and labels to numpy arrays
images = np.array(images)
labels = np.array(labels)

# split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# define the CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(200, 200, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(4, activation='softmax')
])

# compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# train the model
model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))
